In [ ]:
#goal: have a table that looks like this
#how to 

'''
|country|first_idx|second_idx|third_idx|

switzerland 1       13         12



////

then incorporate weights



tables DONE:
* salary (242) (biggest list - treated as master)
* safety (163); 92 404s? need to inspect
* life expectancy (183)
* days off (194)

to-do:


------Q------

need to do manual inspection for some countries (e.g. safety - Luxembourg)



----Notes----
NaN replaced with 404







In [ ]:
#should the profession script be run here?

In [1]:
import pandas as pd

sal = pd.read_csv("salary.csv", delimiter = ';', header = None, index_col = False)
sal.rename(columns = {0: 'salary idx', 1: 'Country', 2: 'Salary', 3: 'Currency', 4: 'Salary[USD]'}, inplace = True)
sal.sort_values(by = 'Salary[USD]', ascending = False, inplace = True)
sal.reset_index(drop=True, inplace=True)  
sal.tail()
sal.dropna(inplace = True)

countries = []
column_list = []
for country in sal['Country']:
    countries.append(country)

master = pd.DataFrame(columns = ["Country", "Salary", "Safety", "Life Expectancy", "Days Off"])
master.head()
master["Country"] = countries

master.head()



#sal.drop(['NaN'])

,Country,Salary,Safety,Life Expectancy,Days Off
0,Switzerland,NaN,NaN,NaN,NaN
1,United States,NaN,NaN,NaN,NaN
2,Denmark,NaN,NaN,NaN,NaN
3,Japan,NaN,NaN,NaN,NaN
4,Luxembourg,NaN,NaN,NaN,NaN


In [2]:
def master_add(column, listname = []): 
    #premise: not adding the values, but the position in the ranking    
    
    for country in countries:
        try:
            value = listname[listname['Country']==country].index.item()
            real_val = listname.at[value, column] # 
            
            value+=1
        except: #Country not found
            value = 404 #bc not found
        
        m_val = master[master['Country'] == country].index.item()   #not in the try block bc master has all the countries
        master.at[m_val, column] = value # real_val
    

In [3]:
#already sorted

saf = pd.read_csv("scrapers/safety.csv", delimiter = ';', header = None, index_col = False)
saf.rename(columns = {0: 'Safety', 1: 'Country', 2: 'Score'}, inplace = True)
saf.drop(columns = "Score", axis = 1, inplace = True)
saf.head()

,Safety,Country
0,1,Iceland
1,2,New Zealand
2,3,Portugal
3,4,Austria
4,5,Denmark


In [4]:
#already sorted

life_exp = pd.read_csv("scrapers/life-exp.csv", delimiter = ';', header = None, index_col = False)
life_exp.rename(columns = {0: 'Life Expectancy', 1: 'Country', 2: 'Predicted Score'}, inplace = True)
life_exp.drop(columns = "Predicted Score", inplace = True)
life_exp.head()



,Life Expectancy,Country
0,1,Japan
1,2,Switzerland
2,3,Singapore
3,4,Australia
4,5,Spain


In [5]:
days_off = pd.read_csv("scrapers/days_off.csv", delimiter = ';', header = None, index_col = False)
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)



days_off.head()

len(days_off)

194

In [6]:
master_add("Salary", sal)
master_add("Safety", saf)
master_add("Life Expectancy", life_exp)
master_add("Days Off", days_off)

In [7]:
master.set_index("Country", inplace = True)
master.head()

,Salary,Safety,Life Expectancy,Days Off
Country,,,,
Switzerland,1,10,2,95
United States,2,128,31,188
Denmark,3,5,27,23
Japan,4,9,1,185
Luxembourg,5,404,13,7


In [8]:
def point_add(category, distr, priority):
    #assign points depending on the type of distribution
    
    
    tr = 10 * distr
    new_col = category + " points"
    if new_col not in column_list:
        column_list.append(new_col)
    master[new_col] = ""
#    print tr
    for country in countries:
        plz = master.loc[country, category]
        if plz <= tr:
#            print country, plz
            pts = (10 * priority)
        elif plz < (2 * tr):
            pts = (9 * priority)
        elif plz < (3 * tr):
            pts = (8 * priority)
        elif plz < (4 * tr):
            pts = (7 * priority)
        elif plz < (5 * tr):
            pts = (6 * priority)
        elif plz < (6 * tr):
            pts = (5 * priority)
        elif plz < (7 * tr):
            pts = (4 * priority)
        elif plz < (8 * tr):
            pts = (3 * priority)
        elif plz < (9 * tr):
            pts = (2 * priority)
        elif plz < (10 * tr):
            pts = priority
    
        else:
            pts = 0
        
        master.at[country, new_col] = pts    
        

In [9]:
point_add("Safety", 1, 2)
point_add("Days Off", 1.4, 1.5)
point_add("Life Expectancy", 1.2, 1.3)
point_add("Salary", 1.1, 1.9)

#can do: based on how important the category is, impact the distr?


In [10]:
master.head() #current

,Salary,Safety,Life Expectancy,Days Off,Safety points,Days Off points,Life Expectancy points,Salary points
Country,,,,,,,,
Switzerland,1,10,2,95,20,6,13,19
United States,2,128,31,188,0,0,10.4,19
Denmark,3,5,27,23,20,13.5,10.4,19
Japan,4,9,1,185,20,0,13,19
Luxembourg,5,404,13,7,0,15,11.7,19


In [11]:
#determine whether a country speaks your language
lan = pd.read_csv("scrapers/languages.csv", delimiter = ';', header = None, index_col = False)

'''
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)
'''

lan.head()

,0,1,2
0,1,Albania,"Albanian (Tosk is the official dialect), Greek"
1,2,Algeria,"Arabic (official), French, Berber dialects"
2,3,Andorra,"Catalán (official), French, Castilian, Portuguese"
3,4,Angola,"Portuguese (official), Bantu and other African..."
4,5,Antigua and Barbuda,"English (official), local dialects"


In [12]:
def lang_add(language, priority):
    lan_col = language + " points"
    if lan_col not in column_list:
        column_list.append(lan_col)
    
    master[lan_col] = ""
    pts = 10 * priority
    lgh = len(lan)

    for no in range(lgh): #determine the right range
        if (language in lan.at[no,2]):
            points = pts 
        else:
            points = 0
           
        #print lan.at[no, 1], lan.at[no, 2], points
        master.at[lan.at[no,1], lan_col] = points    
        
        
        
    for idx, row in master.iterrows():
        if(row[lan_col] == ""):
            row[lan_col] = 0
    #not every country gets points assigned



In [13]:
lang_add("Polish", 1.5)

In [13]:
#master.head(50)
#master.drop(["SUM"], axis = 1, inplace = True)
lang_add("English", 1.8)
lang_add("German", 1.2)

In [31]:
master.sort_values(by = "SUM", inplace =True, ascending = False)

In [38]:
master.drop(["SUM"], axis = 1, inplace = True)

In [14]:
#lang_add("German", 1.6) - case when language already added
master["SUM"] = master[column_list].sum(axis=1)
master.sort_values(by = "SUM", inplace =True, ascending = False)
master.head()

,Salary,Safety,Life Expectancy,Days Off,Safety points,Days Off points,Life Expectancy points,Salary points,English points,German points,SUM
Country,,,,,,,,,,,
Iceland,11,1,6,26,20,13.5,13,19,18,12,95.5
Denmark,3,5,27,23,20,13.5,10.4,19,18,12,92.9
New Zealand,10,2,17,67,20,9,11.7,19,18,0,77.7
Australia,16,13,4,73,18,7.5,13,17.1,18,0,73.6
Austria,31,4,18,16,20,13.5,11.7,15.2,0,12,72.4


In [18]:
master.sort_values(by = "SUM", inplace = True, ascending = True)
master.fillna(0, inplace = True)
master.head()

,Salary,Safety,Life Expectancy,Days Off,Safety points,Days Off points,Life Expectancy points,Salary points,English points,German points,SUM
Country,,,,,,,,,,,
Western Sahara (proposed state),0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
Vatican City (Holy See),0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
American Samoa,238,404,404,404,0,0.0,0.0,0.0,0.0,0.0,0.0
São Tomé and Príncipe,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
Puerto Rico,112,404,404,170,0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
master.sort_values(by = "SUM", inplace = True, ascending = False)
master.head(30)

,Salary,Safety,Life Expectancy,Days Off,Safety points,Days Off points,Life Expectancy points,Salary points,English points,German points,SUM
Country,,,,,,,,,,,
Iceland,11,1,6,26,20,13.5,13.0,19.0,18.0,12.0,95.5
Denmark,3,5,27,23,20,13.5,10.4,19.0,18.0,12.0,92.9
New Zealand,10,2,17,67,20,9.0,11.7,19.0,18.0,0.0,77.7
Australia,16,13,4,73,18,7.5,13.0,17.1,18.0,0.0,73.6
Austria,31,4,18,16,20,13.5,11.7,15.2,0.0,12.0,72.4
Switzerland,1,10,2,95,20,6.0,13.0,19.0,0.0,12.0,70.0
Singapore,6,7,3,159,20,0.0,13.0,19.0,18.0,0.0,70.0
Canada,22,6,12,117,20,3.0,13.0,15.2,18.0,0.0,69.2
Ireland,40,12,19,82,18,7.5,11.7,13.3,18.0,0.0,68.5


In [15]:
print column_list

['Safety points', 'Days Off points', 'Life Expectancy points', 'Salary points', 'English points', 'German points']


In [30]:
master.drop(["Polish points"], axis =1, inplace = True)
#master.sort_values(by ="SUM", inplace =True, ascending = False)
#master.head(20)

#cool! now work on the languages

In [32]:
master["SUM"] = master[column_list].sum(axis=1)
master.head()

KeyError: "['Polish points'] not in index"

In [12]:
#sum computation - move to the end
#master["SUM"] = ""

'''
sum_points = 0
master["SUM"] = master[column_list].sum(axis = 1)
for country in countries:
    sum_points = master.loc[country, "Safety points"] + master.loc[country, "Days Off points"] + master.loc[country, "Life Expectancy points"] + master.loc[country, "Salary points"]# + master.loc[country, "English points"]
    sum_points = sum_points + 
    master.loc[country,"SUM"] = sum_points
master.sort_values(by = "SUM", ascending = False, inplace = True)
master.head()
'''



#master["SUM"] = master[column_list].sum(axis=1)
#pts_df["SUM"] = pts_df.astype(float).sum(axis =1)

# i think the prob is that some entries don't have assigned points at all

['Safety points', 'Days Off points', 'Life Expectancy points', 'Salary points']


/Users/mikowhy/opt/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Safety points,Days Off points,Life Expectancy points,Salary points
Country,,,,
Switzerland,20,6.0,13.0,19.0
United States,0,0.0,10.4,19.0
Denmark,20,13.5,10.4,19.0
Japan,20,0.0,13.0,19.0
Luxembourg,0,15.0,11.7,19.0
Singapore,20,0.0,13.0,19.0
Norway,16,12.0,11.7,19.0
Hong Kong,0,0.0,0.0,19.0
United Kingdom,12,6.0,11.7,19.0


In [35]:
pts_df.head()

,Safety points,Days Off points,Life Expectancy points,Salary points,English points
Country,,,,,
Switzerland,20,6.0,13.0,19.0,0
United States,0,0.0,10.4,19.0,15
Denmark,20,13.5,10.4,19.0,15
Japan,20,0.0,13.0,19.0,0
Luxembourg,0,15.0,11.7,19.0,0


In [210]:
taxes = pd.read_csv("scrapers/taxes.csv", delimiter = ';', header = None, index_col = False)
taxes.sort_values(by = 2, ascending = True, inplace = True)
taxes.drop(columns = 0, inplace = True)
taxes.reset_index(drop = True, inplace = True)
taxes.rename(columns = {1: "Country", 2: "Income Tax"}, inplace = True)
taxes.head(10)

,Country,Income Tax
0,United Arab Emirates,0.0
1,Bahamas,0.0
2,Bahrain,0.0
3,Bermuda,0.0
4,Brunei,0.0
5,Saudi Arabia,0.0
6,Kuwait,0.0
7,Oman,0.0
8,Qatar,0.0
9,Cayman Islands,0.0


In [157]:
master_add("Income Tax", taxes)
master.head()

KeyError: 'Country'

In [ ]:
'''
ideas for distribution:
1. points
    100 points in total
    by choosing which category is more important -> that category gets more points
    then in each category can assign the points by choice:
            eg. if max 10 points to distribute
                    1-10: 10
                    10-20: 8
                    
                    .. etc (Nan's get 0 - but need to make sure which ones are actually NaN - manual inspection)
                    
                    if some category worth X points:
                    1 - 10: x
                    10 - 20: 0.9x
                    20 - 30: 0.8x
                    30 - 40: 0.7x
                    40 - 50: 0.6x
                    50 - 100: 0.5x
                    100 - 150: 0.3x
                    150+ : 0?
                    
            but it should be different dependent on the category
            e.g. the differences in the income tax are not significant between 110th and 150th i think
                    

next phases of work:
    * reading the remaining csv's
        distance from home is still an important functionality that's missing
    * experimenting with the basic functionality
            need to inspect manually the values first, before assigning the points
        btw remember that salary is the only column that changes - the rest can just be generated and re-loaded
    --> writing a GUI!
    
    
    
to fix/edit: 
    -case when langauge / feature already added - make sure you ignore then
    - when dropping a feature, needs to be removed from the col_list as well
    - 0 & 404 when country not found - decide on one
'''

In [15]:
#countring the ones that are 404
saf_null = master[master["Safety"] == 404].count()
life_exp_null = master[master["Life Expectancy"] == 404].count()
days_off_null = master[master["Days Off"] == 404].count()

print saf_null, life_exp_null, days_off_null


Country            92
Salary             92
Safety             92
Life Expectancy    92
Days Off           92
dtype: int64 Country            68
Salary             68
Safety             68
Life Expectancy    68
Days Off           68
dtype: int64 Country            61
Salary             61
Safety             61
Life Expectancy    61
Days Off           61
dtype: int64


In [11]:
print column_list

['Safety points', 'Days Off points', 'Life Expectancy points', 'Salary points']
